<a href="https://colab.research.google.com/github/trandlong/MarketingResearch/blob/master/Twitter_connection_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install tweepy --upgrade

In [2]:
import tweepy as tw
import numpy as np
import pandas as pd
import requests
import os
import json
from datetime import date, datetime, timedelta, timezone

In [4]:
datetime.utcnow() -timedelta(days = 7)

datetime.datetime(2022, 3, 20, 23, 51, 8, 842866)

In [5]:
datetime.now(timezone.utc)

datetime.datetime(2022, 3, 27, 23, 51, 8, 855745, tzinfo=datetime.timezone.utc)

In [6]:

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = my_bearer_token


# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields


def create_url(keyword, start_time = datetime.now(timezone.utc)-timedelta(days=7),  max_results = 50):
    """ 
    create search URL
    defaule time  7 days prior
    default results = 50
    """

    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    start_time =  start_time.isoformat('T')
    query_params = {'query': keyword,

                    'max_results': max_results,
                    'start_time':start_time,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id,attachments.media_keys',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'media.fields':'duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width,alt_text',
                    'next_token': {}}
    return (search_url, query_params)

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(search_url, params):

    """
    connect end point to URL
    """
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def search_tweet(keyword, start_time = datetime.now(timezone.utc)-timedelta(days=7),  max_results = 50):

    search_url, query_params = create_url(keyword, start_time, max_results)
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response
    #(json.dumps(json_response, indent=4, sort_keys=True))




In [7]:
query_pepsi= '#pepsi has:images lang:en'

pepsi_tweets  = search_tweet(query_pepsi )


In [8]:
#populate tweets data
tweets_df  = pd.DataFrame(columns=['media_key','author_id', 'conversation_id','tweet_id','lang','like_count','quote_count','reply_count','retweet_count','reply_settings','source', 'text']) #pepsi_tweets['data']
for i in range(len(pepsi_tweets["data"])):  #iterate over the tweet
    try:
      media_keys = pepsi_tweets["data"][i]["attachments"]['media_keys'] #grab media key of the attachment
      author_id = pepsi_tweets["data"][i]["author_id"]
      conversation_id = pepsi_tweets["data"][i]["conversation_id"]
      tweet_id = pepsi_tweets["data"][i]["id"]
      lang = pepsi_tweets["data"][i]["lang"]
      reply_settings = pepsi_tweets["data"][i]["reply_settings"]
      source = pepsi_tweets["data"][i]["source"]
      tweet_text = pepsi_tweets["data"][i]["text"]
      like_count =pepsi_tweets["data"][i]["public_metrics"]['like_count']
      quote_count =pepsi_tweets["data"][i]["public_metrics"]['quote_count']
      reply_count =pepsi_tweets["data"][i]["public_metrics"]['reply_count']
      retweet_count =pepsi_tweets["data"][i]["public_metrics"]['retweet_count']
      tweets_df = tweets_df.append(pd.DataFrame({'media_key':media_keys,
                                                            'author_id':author_id,
                                                            'conversation_id':conversation_id,
                                                            'tweet_id':tweet_id,
                                                            'lang':lang,
                                                            'reply_settings':reply_settings,
                                                            'source':source,
                                                            'text':tweet_text,
                                                            'like_count':like_count,
                                                            'quote_count':quote_count,
                                                            'reply_count':reply_count,
                                                            'retweet_count':retweet_count,
                                                            'reply_settings':reply_settings,

                                                

                                  }))
      tweets_df = tweets_df.reset_index(drop=True)
    except:
      pass

In [9]:
pepsi_tweets['includes']['users'][0]["id"]

'1471161157880889344'

In [10]:
#populate user data
users_df = pd.DataFrame(columns=[ 'user_created_at','description', 'author_id','name','tweet_count','username','verified','followers_count','following_count',  'listed_count' ])#pepsi_tweets['includes']['users'][0]

for i in range(len(pepsi_tweets['includes']['users'])):  #iterate over the tweet
    try:
      user_created_at =pepsi_tweets['includes']['users'][i]["created_at"]
      description = pepsi_tweets['includes']['users'][i]["description"]
      author_id = pepsi_tweets['includes']['users'][i]["id"]
      name = pepsi_tweets['includes']['users'][i]["name"]
      tweet_count =pepsi_tweets['includes']['users'][i]["public_metrics"]['tweet_count']
      username = pepsi_tweets['includes']['users'][i]["username"]
      verified = pepsi_tweets['includes']['users'][i]["verified"]
      following_count =pepsi_tweets['includes']['users'][i]["public_metrics"]['following_count']
      followers_count =pepsi_tweets['includes']['users'][i]["public_metrics"]['followers_count']
      listed_count =pepsi_tweets['includes']['users'][i]["public_metrics"]['listed_count']

      to_add = pd.DataFrame({'user_created_at':user_created_at,
                              'description':description,
                              'author_id':author_id,
                              'tweet_count':tweet_count,
                              'name':name,
                              'username':username,
                              'verified':verified,
                              'following_count':following_count,
                              'followers_count':followers_count,
                              'listed_count':listed_count},index = [1])
      users_df = users_df.append(to_add)
      users_df = users_df.reset_index(drop=True)
    except:
      pass

In [11]:
users_df

,user_created_at,description,author_id,name,tweet_count,username,verified,followers_count,following_count,listed_count
0,2021-12-15T16:52:23.000Z,Absorb || Express - Life Lover. Music Producer...,1471161157880889344,Prod.by_V!BZ,6961,prodby_vibz,False,341,142,4
1,2011-04-21T12:44:24.000Z,"The exact dev you need. dart, reactnative, jav...",285589811,Benjamin Atwiine,418,BenTheCodeGuru,False,194,149,5
2,2010-12-14T14:49:18.000Z,The Can Museum is an international collaborati...,226579457,Can Museum,1798,CanMuseum,False,254,329,9
3,2008-12-15T16:26:23.000Z,If you’re looking for hot takes and cold Pepsi...,18139619,Pepsi,73465,pepsi,True,3086778,38110,6167
4,2018-04-15T20:23:05.000Z,,985614510034694144,Vanessa Etim,115826,VanesEtim,False,1501,34,15
5,2008-09-30T17:00:53.000Z,The World's largest online retailer of streetw...,16530463,Karmaloop.com,47735,Karmaloop,True,372875,73,1440
6,2022-03-26T06:54:46.000Z,I'm a Nigerian,1507611927257702406,Abr. David,17,davbr65,False,1,1,0
7,2020-11-17T05:51:41.000Z,It's definitely impossible to fall for some ot...,1328576485356150786,_yizhanislove_,81,yiling_laozuu_,False,3,113,0
8,2020-01-05T12:43:22.000Z,"koi Hamse nafrat kre aise ho nhi sakta, aur ag...",1213803048260104192,ⓇअनामिकाⓋ🇮🇳💞 (Salmaniacs),49260,Anokhiajnabi,False,727,575,0
9,2010-07-09T12:25:59.000Z,Extraordinary| MuseumPiece| Friendly| Taurean|...,164658749,Ann Mathew,80481,bindiyannsmitha,False,3593,727,23


In [12]:
media_df = pd.DataFrame(columns=['media_key','media_type','height', 'width','url']) #pepsi_tweets['includes']['media'][0]
for i in range(len(pepsi_tweets['includes']['media'])):  #iterate over the tweet
    try:
      media_key =pepsi_tweets['includes']['media'][i]["media_key"]
      media_type = pepsi_tweets['includes']['media'][i]["type"]
      height = pepsi_tweets['includes']['media'][i]["height"]
      width = pepsi_tweets['includes']['media'][i]["width"]
      url = pepsi_tweets['includes']['media'][i]["url"]
   
      to_add = pd.DataFrame({'media_key':media_key,
                              'media_type':media_type,
                              'height':height,
                              'height':tweet_count,
                              'width':width,
                              'url':url,
                            },index = [1])
      media_df = media_df.append(to_add)
      media_df = media_df.reset_index(drop=True)
    except:
      pass

In [13]:
media_df

,media_key,media_type,height,width,url
0,3_1507716582184730637,photo,853,632,https://pbs.twimg.com/media/FOx8PkcWQA0NBE3.png
1,3_1508136002098970633,photo,853,554,https://pbs.twimg.com/media/FO35tBYWQAkL71i.jpg
2,3_1508133032015937545,photo,853,2048,https://pbs.twimg.com/media/FO33AI9XwAk16dz.jpg
3,3_1508096589172293640,photo,853,1866,https://pbs.twimg.com/media/FO3V24yXwAgX3Kj.jpg
4,3_1507652837593202688,photo,853,2048,https://pbs.twimg.com/media/FOxCRJVaQAAqkGS.jpg
5,3_1506538280845975556,photo,853,2048,https://pbs.twimg.com/media/FOhMlZ5aIAQkVb-.jpg
6,3_1507633707808550917,photo,853,2048,https://pbs.twimg.com/media/FOww3pVVcAUvcL1.jpg
7,3_1507986491800276992,photo,853,720,https://pbs.twimg.com/media/FO1xuYHX0AAZnlm.jpg
8,3_1507905217806974977,photo,853,2048,https://pbs.twimg.com/media/FO0nzm5XwAEQ7SG.jpg
9,3_1507828500970946562,photo,853,972,https://pbs.twimg.com/media/FOziCGaVQAIvPX0.jpg


In [14]:
# save
#tweets_df.to_csv("test_50tweets_data.csv")
#!cp test_50tweets_data.csv "drive/My Drive/"